## SparkSQL로 Spark 데이터 추출 및 전처리
- [Github Page for Tutorial](https://github.com/songhunhwa/songhunhwa.github.com/tree/master/tutorial/tutorial_01)
- SQLContext 생성
- DataFrame 생성 및 추출
- SQL 전처리 및 분석
  - temporary table 생성
  - `select`, `join`, `groupby-agg-alias`, `describe`, `show`

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 198 kB 62.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=a3f57f4a95f15d28109d01a79f9c1f5b92acff382d13c4eb84dbd21dd2fd794f
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
# 모듈 import
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

In [4]:
sc = SparkContext()
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


> **캘리포니아 집값 정보** CSV 파일을 읽어와 봅시다.

`sqlContext`객체로 읽어오기!

In [7]:
# read the csv with library
df = sqlContext.read.format('com.databricks.spark.csv')\
					.options(header='true', inferSchema='true')\
					.load('/content/sample_data/california_housing_train.csv')\
					.cache()

In [9]:
df.take(3)

[Row(longitude=-114.31, latitude=34.19, housing_median_age=15.0, total_rooms=5612.0, total_bedrooms=1283.0, population=1015.0, households=472.0, median_income=1.4936, median_house_value=66900.0),
 Row(longitude=-114.47, latitude=34.4, housing_median_age=19.0, total_rooms=7650.0, total_bedrooms=1901.0, population=1129.0, households=463.0, median_income=1.82, median_house_value=80100.0),
 Row(longitude=-114.56, latitude=33.69, housing_median_age=17.0, total_rooms=720.0, total_bedrooms=174.0, population=333.0, households=117.0, median_income=1.6509, median_house_value=85700.0)]

> `printSchema()`를 통해 구조를 확인합니다.

In [10]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



`df.count()`를 통하여 개수를 셉니다.

In [11]:
df.count()

17000

> tmp table로 변환합니다.
 
> tmp란? temporary의 약자로서, 1회 조회용을 위한 데이터 생성

In [12]:
# convert the df to tmp table (as if it's in database)
df.registerTempTable("df_tmp")

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


> SQL문을 통하여 인구가 10000명 이상인 지점의 위도, 경도, 인구와 수입 중앙값을 새로운 데이터프레임에 할당시켜 봅시다.

In [13]:
# extract data from table with sql
df1 = sqlContext.sql("select longitude, latitude, population, median_income from df_tmp where population>10000")

In [14]:
df1.take(3)

[Row(longitude=-117.12, latitude=33.52, population=13251.0, median_income=4.5841),
 Row(longitude=-117.42, latitude=33.35, population=35682.0, median_income=2.5729),
 Row(longitude=-117.61, latitude=34.1, population=10323.0, median_income=3.6033)]

In [15]:
df1.count()

19

> temporary table로부터 다시 df2로 할당하여 Lazy execution 실습을 하여 봅시다.

In [35]:
## Lazy Execution
df2 = sqlContext.sql("select * from df_tmp")

In [36]:
df2.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
+---------+--------+----

> 위도별 방 개수의 최솟값을 집계하여 봅시다!
> `groupby`, `agg`, `alias`를 이용합니다

In [38]:
df2_lat_to_rooms = df2.groupby("latitude").agg(min("total_rooms").alias("min_rooms"))
df2_lat_to_rooms.show()

+--------+---------+
|latitude|min_rooms|
+--------+---------+
|   35.17|    116.0|
|   38.61|    438.0|
|   37.81|     12.0|
|   40.11|   1631.0|
|   40.53|   1127.0|
|   37.23|   1656.0|
|   39.42|    411.0|
|   40.94|   1452.0|
|   36.27|   1494.0|
|    37.1|    828.0|
|   38.93|    231.0|
|   35.15|   7922.0|
|   38.68|     67.0|
|   41.46|   1277.0|
|   35.34|    164.0|
|   35.38|    346.0|
|   37.29|    436.0|
|   40.14|   1950.0|
|   41.78|   2918.0|
|   39.97|   1468.0|
+--------+---------+
only showing top 20 rows



df1 옆에 `df2_lat_to_rooms` 을 붙여봅시다.

In [39]:
df2_join = df1.join(df2_lat_to_rooms, "latitude", "left")
df2_join.show()

+--------+---------+----------+-------------+---------+
|latitude|longitude|population|median_income|min_rooms|
+--------+---------+----------+-------------+---------+
|   33.52|  -117.12|   13251.0|       4.5841|    426.0|
|   33.35|  -117.42|   35682.0|       2.5729|   1586.0|
|    34.1|  -117.61|   10323.0|       3.6033|    299.0|
|   33.89|  -117.74|   16122.0|       7.4947|     63.0|
|   34.01|  -117.75|   10450.0|       6.1287|    352.0|
|   34.03|  -117.78|   15507.0|       6.0191|    213.0|
|   34.04|  -117.87|   15037.0|       6.6288|     18.0|
|   33.96|  -117.88|   10988.0|       5.5469|    124.0|
|   34.68|  -118.09|   10493.0|       4.0211|    552.0|
|   34.57|   -118.1|   11973.0|       3.3086|   1054.0|
|    34.4|  -118.46|   10475.0|        4.542|   1112.0|
|   34.16|  -118.78|   12873.0|       8.0137|     54.0|
|   34.26|   -118.9|   11956.0|       6.9712|    224.0|
|    34.7|  -120.59|   12427.0|       2.7377|   1943.0|
|   38.47|   -121.4|   10329.0|       4.3658|   

> 위도, 경도에 따른 최소 방 개수를 집계하여 봅시다.

In [42]:
df2_join1 = df2_join.groupby("latitude", "longitude").agg(count("min_rooms").alias("tude_to_rooms"))

In [45]:
df2_join1.show(5)

+--------+---------+-------------+
|latitude|longitude|tude_to_rooms|
+--------+---------+-------------+
|    34.1|  -117.61|            1|
|   36.69|  -121.61|            1|
|   33.35|  -117.42|            1|
|   33.89|  -117.74|            1|
|   36.72|  -121.68|            1|
+--------+---------+-------------+
only showing top 5 rows



In [43]:
df2_join1.describe().show()

+-------+------------------+------------------+-------------+
|summary|          latitude|         longitude|tude_to_rooms|
+-------+------------------+------------------+-------------+
|  count|                19|                19|           19|
|   mean|34.932631578947365|-119.0784210526316|          1.0|
| stddev|1.4856605044568298| 1.757476926220892|          0.0|
|    min|             33.35|           -121.92|            1|
|    max|             38.47|           -117.12|            1|
+-------+------------------+------------------+-------------+



In [28]:
# Pandas로 변환하기
df2_pd = df2.toPandas()
df2_pd.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [27]:
df2_pd.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


# other functions
fillna()<br>
dropDuplicates()<br>
drop()<br>
distinct()<br>
countDistinct()<br>
withColumn()<br>
withColumnRenamed()<br>
pivot()<br>
sort()<br>
collect_list()<br>
collect_set()<br>
get_json_object()<br>
from_unixtime()<br>
to_date()<br>
sample(False, 0.1, 123) <br>
cube()<br>
cache()<br>